### Tamagotchi MDP
pam o.p. 2018
Not a real tamagotchi, but a simple version of one

In [1]:
# imports
from random import randint
import numpy as np
import random
import datetime # for limiting calculation to wall clock time
from math import log, sqrt
import copy

In [26]:
# tamagotchi class adapted from https://github.com/bitterfly/tamagotchi/blob/master/tamagotchi/core/tamagotchi.py

class Tamagotchi:
    def __init__(self):
        self.stats = {"food": 100, "happiness": 100, "hygiene": 100,
                 "health": 100, "energy": 100}
        self.is_sleeping = False
        self.is_dead = False
        self.is_playing = False
        self.is_sick = False
        self.number_of_poo = 0
        self.money = 0
        self.time = 0
        self.in_store = False

    #Makes sure the statistic isn't below 0 or above 100
    def constrain(self, value):
        value = min(100, value)
        value = max(0, value)
        return value

    #Constrains all the stats
    def constrain_stats(self):
        for statistic, value in self.stats.items():
            self.stats[statistic] = self.constrain(value)

    #Takes a dictionary with statistics and adds each value
    #to the tamagotchi statistics
    def apply(self, item):
#         for statistic, value in self.stats.items():
#             self.stats[statistic] += item[statistic]
        self.stats[item['stats']] += item['effect']
        self.constrain_stats()

#     #Takes a statistic and decreases it to zero in "full hours" time
    def decrease_to_minimum(self, statistic, full_hours, time_given):
        self.stats[statistic] -= np.ceil( (time_given * 10) / (full_hours * 36) ) 
#     def decrease_to_minimum(self, statistic, full_hours, time_given):
#         self.stats[statistic] -= int(full_hours/2)

#     #Takes a statistic and increases it to max in "full hours" time
    def increase_to_maximum(self, statistic, full_hours, time_given):
        self.stats[statistic] += np.ceil( (time_given * 10) / (full_hours * 36) ) 
#     def increase_to_maximum(self, statistic, full_hours, time_given):
#         self.stats[statistic] += int(full_hours)

    #Generates random sickness and poo
    def random_event(self):
        if (not self.is_playing and not self.is_sleeping):
            random_number = randint(0, 600)
            if random_number == 0:
                self.is_sick = True
            if random_number == 1:
                self.number_of_poo = min (self.number_of_poo + 1, 4)

    #Used in mainwindow - removes sickness
    def cure(self):
        self.is_sick = False

    #The function witch decreases all the stats every second
    #or is called when tamagotchi is sleeping
    def second_pass(self, seconds=1):
        "Докато спиш всички статове падат за 8 часа, освен сънят, който се възстановява"
        "As long as you sleep, all the stats fall for 8 hours, except for the sleep that is recovering"
        self.time += 1 
        
        if self.is_sleeping:
            self.increase_to_maximum("energy", 8, seconds)
            self.decrease_to_minimum("happiness", 12, seconds)
            self.decrease_to_minimum("hygiene", (12 + 2*self.number_of_poo), seconds)
            self.decrease_to_minimum("food", 12, seconds)
            
            if self.stats["energy"] > 70:
                self.is_sleeping = False
        else:
            "Докато играеш, повечето статистики падат по-бързо."
            "As you play, most stats fall faster"
            if self.is_playing:
                self.decrease_to_minimum("energy", 3, seconds)
                self.decrease_to_minimum("hygiene", 3, seconds)
                self.decrease_to_minimum("food", 4, seconds)
                self.increase_to_maximum("happiness", 1, seconds)
                self.money += seconds # pam added
            else:
                self.decrease_to_minimum("energy", 4, seconds)
                self.decrease_to_minimum("hygiene", 4 / (self.number_of_poo + 1), seconds)
                self.decrease_to_minimum("food", 4, seconds)
                self.decrease_to_minimum("happiness", 4, seconds)

        if (self.stats["happiness"] <= 50 or
               self.stats["hygiene"] <= 50):
                self.is_sick = True
        # add that poo around acts like sickness?
        if self.is_sick:
            self.decrease_to_minimum("health", 3, seconds)

        self.constrain_stats()

        self.random_event()

        if (self.stats["food"] == 0 or self.stats["health"] == 0):
            self.is_dead = True
        
        #add (?) that if energy is <=20 then goes to sleep
        if self.stats["energy"] <=20:
            self.is_sleeping = True
#         elif self.stats["energy"]

    #The function is called when an item is chosen and
    #applies its statistics
    def buy_item(self, item):
        self.apply(item)
        self.money -= item["price"]
        return self
    
    def print_tama(self):
        print("Time:",self.time," seconds elapsed")
        if self.is_dead:
            print("Tama is dead!")
    
        print("TAMA STATS: ", self.stats)
        if self.is_sleeping:
            print("Tama is asleep.")
        else:
            print("Tama is awake.")
            if self.is_playing:
                print("Tama is playing.")
            if self.is_sick:
                print("Tama is sick.")
        print("Number of poo:",self.number_of_poo)
        print("Money:",self.money)
        if self.in_store:
            print("You're in the store, so you can buy an item if you want.")
        else:
            print("You're not in the store, so you can play, do nothing, or travel to the store.")
                
        print("\n")

In [27]:
# game class
# gaining money to buy items in store can be simple time cost/money reward exchange
# agent decides when to gain money, and then which items to buy/apply to tamagotchi
class Tamagotchi_Game():
    def __init__(self):
#         self.tamagotchi = tamagotchi
#         self.time = 0 #seconds
        self.time_passing = True
#         self.gamelength = 100000
#         self.in_store = False
    
    # i need to wrap all the tamagotchi fxns into the structure of the game.
    # e.g. the items that can be bought at the store, and how they cause .apply() or .cure() to the tama
    # depending on the item
    
#     # start game 
#     def start(self,state):
#         self.time_passing = True
# #         self.gamelength = gamelength
#         return state
        
    # currently, this is sorta useless; don't need the pomdp to be able to pause the game
    # but would be useful for a human playing the game to take breaks
#     def pause(self):
#         self.time_passing = False
#         return self.tamagotchi
        
        
    # given current tamagotchi "state" and action "action", what would the next tama state be?
    def next_state(self, state, action):
#         state_copy = copy(state)
        # if time is passing
        if self.time_passing:
            # time passes - currently same amount for each action but could change this 
#             self.tamagotchi.second_pass() # does this need to be state.second_pass()
#             self.time +=1
            
            if state.in_store:
                state.is_playing = False
                state.second_pass() # does this need to be state.second_pass()
                
                if action == 'nothing':
                    state.in_store = False
                    return state
                elif action == 'coffee':
                    return state.buy_item({"stats":"energy","price":3,"effect":5})
                elif action == 'snack':
                    return state.buy_item({"stats":"food","price":3,"effect":5})
                elif action == 'clean':
                    state.number_of_poo =0
                    return state.buy_item({"stats":"hygiene","price":5,"effect":5})
                elif action == 'medicine':
                    state.cure()
                    return state.buy_item({"stats":"health","price":8,"effect":10})
                else:
                    raise Exception('Invalid action while in store')
            
            if action == 'play':
                state.is_playing = True
                state.second_pass() # does this need to be state.second_pass()
                # if the tama is sleeping, playing won't earn any money - tama needs to be awake
                return state
            
            # "walk" to the store (takes one turn)
            if action == 'store':
                state.is_playing = False
                state.in_store = True
                state.second_pass()
                return state
            
            if action == 'nothing':
                state.is_playing = False
                state.second_pass()
                return state
                
        else: # if paused
            return state
            
                
        
    # Take a sequence of game states representing the full game history, and return the full list
    # of actions that are legal actions
    def legal_actions(self, state_history): 
        # this method is important for MDPs/games where the actions might change based on the game state
        # for instance, in checkers, if player put a piece on square A4, no other piece can move to A4
        
        # Should the MCTS store as state history the tamagotchi at each time point? so saving n tamagotchi
        # objects for n timepoints... so then legal_actions could just take recent state and might depend
        # on whether the tamagotchi is alive or dead, asleep or not, sick or not, etc... 
        # but i don't think this is a problem that depends on prev states (just need to call self.tamagotchi)'
        tama = state_history[-1]

        if self.time_passing and not tama.is_dead:
            if tama.in_store:
                if tama.money >= 8:
                    return ['coffee','snack','clean','medicine','nothing']
                elif tama.money >= 5:
                    return ['coffee','snack','clean','nothing']
                elif tama.money >= 3:
                    return ['coffee','snack','nothing']
                else: # too poor to shop :(
                    return ['nothing']
            else: 
                return ['play','store','nothing']
        else:
            return []
    
    # Should this be the length of state_history (how long tama alive for)? or is it trial by trial rwd?
    def reward(self, state_history):
        done = False
        if state_history[-1].is_dead:
            reward = copy.copy(state_history[-1].time) # you killed the tama :'(
            done = True
        else:
            # reward is how long you kept the tama alive
            reward = 0

        return reward, done
    

In [29]:
tama = Tamagotchi()
game = Tamagotchi_Game()
# s = game.start(tama)
tama.print_tama()

tama = game.next_state(tama,'play')
tama = game.next_state(tama,'play')
tama = game.next_state(tama,'play')
tama = game.next_state(tama,'play')
tama = game.next_state(tama,'store')
tama.print_tama()

game.reward([tama])

Time: 0  seconds elapsed
TAMA STATS:  {'health': 100, 'hygiene': 100, 'happiness': 100, 'food': 100, 'energy': 100}
Tama is awake.
Number of poo: 0
Money: 0
You're not in the store, so you can play, do nothing, or travel to the store.


Time: 5  seconds elapsed
TAMA STATS:  {'health': 100, 'hygiene': 95.0, 'happiness': 99.0, 'food': 95.0, 'energy': 95.0}
Tama is awake.
Number of poo: 0
Money: 4
You're in the store, so you can buy an item if you want.




(0, False)

In [30]:
class MCTS(object):
    
    def __init__(self, tamagotchi_game, **kwargs):
        self.game = tamagotchi_game
        self.states = []
        seconds = kwargs.get('time',30)
        self.calculation_time = datetime.timedelta(seconds=seconds)
        self.max_moves = kwargs.get('max_moves')
        self.C = kwargs.get('C', 1.4)
        self.rewards = {}
        self.plays = {}
        
    # Take a game state and append it to the history
    def update(self,state):
        self.states.append(state)
        
    # AI calculates best move and returns it
    def get_play(self):
        self.max_depth = 0
        state = copy.deepcopy(self.states[-1]) # last state
        legal = self.game.legal_actions(self.states[:])
        
        if legal == []:
            return
        if len(legal)==1:
            return legal[0]
        
        games = 0 # counter for number of games simulated (e.g., number of times "run_sim" is called)
        
        # begin the tree! keep track of time so do not exceed computation time limit
        begin = datetime.datetime.utcnow()
        while (datetime.datetime.utcnow() - begin) < self.calculation_time:
            self.run_simulation()
            games+=1
        # may need for debugging:
        # self.print_tree()
        
        # makes [(state, action1),...,(state, action_n)] for n legal actions
        moves_states = [(state, a) for a in legal]
        
        # display number of calls of 'run_simulation' and the time elapsed
        print('Num sims run:',games, '  Time elapsed:', datetime.datetime.utcnow() - begin)
        
        # pick the move with the highest average reward
        percent_wins, move = max((self.rewards.get((s,a), 0) / self.plays.get((s,a), 1), a)
                                for s, a in moves_states)
        
        
        #### $$$ ****         WORK ON THIS       ****** $$$ #####
        # display the stats for each possible play
        # or maybe not if there are lotta actions
        # **** also this might be wrong, reconsider this code for the tamagotchi
        for x in sorted(
            ((100 * self.rewards.get((s,a), 0) / self.plays.get((s,a), 1),
             self.rewards.get((s,a), 0), self.plays.get((s,a), 0), 
             a)
             for s,a in moves_states),
            reverse=True
        ):
            print("{3}: {0:.2f}% ({1} / {2})".format(*x))
        
        # print the max depth tree search
        print("Maximum depth search:", self.max_depth)
        
        # return the move chosen
        return move
        
        
#     # this don't do anything yet
#     def print_tree(self):
#         board = self.game.tamagotchi_game
#         pass
    
    # play out a "random" game from the current position, then update stats with result
    def run_simulation(self):
        plays, rewards = self.plays, self.rewards
        
        visited_qs = set()
        states_copy = copy.deepcopy(self.states[:])
        state = states_copy[-1]
        
        expand = True # you only expand once #YOEO
        for t in range(self.max_moves):
            legal = self.game.legal_actions(states_copy)  # get valid actions
            if legal == []:
                # THE TAMA IS DEAD
                reward, done = self.game.reward(states_copy)
                break
            moves = [(state, a) for a in legal]
            
            if all(plays.get((s,a)) for s,a in moves):
                # if we have statistics on all legal moves, use them.
                # upper confidence bound (UCB) algorithm
#                 print("UCB choice")
                ## *** might need to change the structure of plays, not sure if this is
                ## doing the right thing re: mean payouts
                log_total = 2.0*log(
                    sum(plays[(s,a)] for s,a in moves)
                )
                
                # value of best
                value, ins = max(
                    ((rewards[(s,a)] / plays[(s,a)]) +
                     self.C * sqrt(log_total / plays[(s,a)]),a)
                    for s,a in moves
                )
            else:
                # if we don't have stats on all legal moves, randomly pick one
#                 print("Random choice")
                action = random.choice(legal) 
                
            # if we are in the expand phase and this is a new state-action pair
            if expand and (state, action) not in self.plays: 
                expand = False # you only expand once so this is it
                self.plays[(state, action)] = 0 # initialize
                self.rewards[(state, action)] = 0
                if t > self.max_depth:
                    self.max_depth = t
                    
            visited_qs.add((state,action)) # add this state as visited
            
            state = self.game.next_state(state,action) # get next state
            states_copy.append(state) # record
            
            reward, done = self.game.reward(states_copy) # compute reward if any
            #print(states_copy, reward, done)
            
            if done: 
                break
        
        # print(visited_states, reward)
        for q in visited_qs: # for each visited state
            if q not in self.plays: # if we don't have stats on this state yet
                continue
            self.plays[q]+=1 # increase plays
            self.rewards[q]+=reward # add up the reward you got
        
        

In [33]:
# Initialize the tamagotchi
realtama = Tamagotchi()
game = Tamagotchi_Game()

print("INITIAL STATE: ")
realtama.print_tama() # initial tamagotchi state

tree = MCTS(game, time=3., C=1.4, max_moves = 100)

action_seq = []
# inp =''

while not realtama.is_dead: # len(action_seq) <= 100: #and not state.is_dead: # play for a certain amount of time (better rule?)
#     state_copy = copy.deepcopy(realtama)
    tree.update(realtama)
    action = tree.get_play()
    action_seq.append(action)
#     next_st = game.next_state(realtama,action)
    print("\nTaking action %s."% (action))


    print("\n Actions taken: ",len(action_seq))
    print("Reward: ",game.reward(tree.states))
    
#     realtama = copy.deepcopy(game.next_state(realtama,action))
    realtama = game.next_state(realtama,action)
    realtama.print_tama()

#     print(state.time)


INITIAL STATE: 
Time: 0  seconds elapsed
TAMA STATS:  {'health': 100, 'hygiene': 100, 'happiness': 100, 'food': 100, 'energy': 100}
Tama is awake.
Number of poo: 0
Money: 0
You're not in the store, so you can play, do nothing, or travel to the store.


Num sims run: 793   Time elapsed: 0:00:03.000447
store: 0.00% (0 / 0)
play: 0.00% (0 / 0)
nothing: 0.00% (0 / 0)
Maximum depth search: 0

Taking action store.

 Actions taken:  1
Reward:  (0, False)
Time: 1  seconds elapsed
TAMA STATS:  {'health': 100, 'hygiene': 99.0, 'happiness': 99.0, 'food': 99.0, 'energy': 99.0}
Tama is awake.
Number of poo: 0
Money: 0
You're in the store, so you can buy an item if you want.



Taking action nothing.

 Actions taken:  2
Reward:  (0, False)
Time: 2  seconds elapsed
TAMA STATS:  {'health': 100, 'hygiene': 98.0, 'happiness': 98.0, 'food': 98.0, 'energy': 98.0}
Tama is awake.
Number of poo: 0
Money: 0
You're not in the store, so you can play, do nothing, or travel to the store.


Num sims run: 797   Tim

Num sims run: 876   Time elapsed: 0:00:03.002011
store: 0.00% (0 / 0)
play: 0.00% (0 / 0)
nothing: 0.00% (0 / 0)
Maximum depth search: 0

Taking action store.

 Actions taken:  23
Reward:  (0, False)
Time: 23  seconds elapsed
TAMA STATS:  {'health': 100, 'hygiene': 77.0, 'happiness': 77.0, 'food': 77.0, 'energy': 77.0}
Tama is awake.
Number of poo: 0
Money: 0
You're in the store, so you can buy an item if you want.



Taking action nothing.

 Actions taken:  24
Reward:  (0, False)
Time: 24  seconds elapsed
TAMA STATS:  {'health': 100, 'hygiene': 76.0, 'happiness': 76.0, 'food': 76.0, 'energy': 76.0}
Tama is awake.
Number of poo: 0
Money: 0
You're not in the store, so you can play, do nothing, or travel to the store.


Num sims run: 870   Time elapsed: 0:00:03.001116
store: 0.00% (0 / 0)
play: 0.00% (0 / 0)
nothing: 0.00% (0 / 0)
Maximum depth search: 0

Taking action store.

 Actions taken:  25
Reward:  (0, False)
Time: 25  seconds elapsed
TAMA STATS:  {'health': 100, 'hygiene': 75.0, 

Num sims run: 1240   Time elapsed: 0:00:03.000533
store: 0.00% (0 / 0)
play: 0.00% (0 / 0)
nothing: 0.00% (0 / 0)
Maximum depth search: 0

Taking action store.

 Actions taken:  47
Reward:  (0, False)
Time: 47  seconds elapsed
TAMA STATS:  {'health': 100, 'hygiene': 53.0, 'happiness': 53.0, 'food': 53.0, 'energy': 53.0}
Tama is awake.
Number of poo: 0
Money: 0
You're in the store, so you can buy an item if you want.



Taking action nothing.

 Actions taken:  48
Reward:  (0, False)
Time: 48  seconds elapsed
TAMA STATS:  {'health': 100, 'hygiene': 52.0, 'happiness': 52.0, 'food': 52.0, 'energy': 52.0}
Tama is awake.
Number of poo: 0
Money: 0
You're not in the store, so you can play, do nothing, or travel to the store.


Num sims run: 1313   Time elapsed: 0:00:03.000588
store: 0.00% (0 / 0)
play: 0.00% (0 / 0)
nothing: 0.00% (0 / 0)
Maximum depth search: 0

Taking action store.

 Actions taken:  49
Reward:  (0, False)
Time: 49  seconds elapsed
TAMA STATS:  {'health': 100, 'hygiene': 51.0

Num sims run: 2089   Time elapsed: 0:00:03.001333
store: 0.00% (0 / 0)
play: 0.00% (0 / 0)
nothing: 0.00% (0 / 0)
Maximum depth search: 0

Taking action store.

 Actions taken:  69
Reward:  (0, False)
Time: 69  seconds elapsed
TAMA STATS:  {'health': 80.0, 'hygiene': 31.0, 'happiness': 31.0, 'food': 31.0, 'energy': 31.0}
Tama is awake.
Tama is sick.
Number of poo: 0
Money: 0
You're in the store, so you can buy an item if you want.



Taking action nothing.

 Actions taken:  70
Reward:  (0, False)
Time: 70  seconds elapsed
TAMA STATS:  {'health': 79.0, 'hygiene': 30.0, 'happiness': 30.0, 'food': 30.0, 'energy': 30.0}
Tama is awake.
Tama is sick.
Number of poo: 0
Money: 0
You're not in the store, so you can play, do nothing, or travel to the store.


Num sims run: 2116   Time elapsed: 0:00:03.000416
store: 0.00% (0 / 0)
play: 0.00% (0 / 0)
nothing: 0.00% (0 / 0)
Maximum depth search: 0

Taking action store.

 Actions taken:  71
Reward:  (0, False)
Time: 71  seconds elapsed
TAMA STATS:  {

Num sims run: 5286   Time elapsed: 0:00:03.000245
store: 0.00% (0 / 0)
play: 0.00% (0 / 0)
nothing: 0.00% (0 / 0)
Maximum depth search: 0

Taking action store.

 Actions taken:  91
Reward:  (0, False)
Time: 91  seconds elapsed
TAMA STATS:  {'health': 58.0, 'hygiene': 9.0, 'happiness': 9.0, 'food': 9.0, 'energy': 31.0}
Tama is asleep.
Number of poo: 0
Money: 0
You're in the store, so you can buy an item if you want.



Taking action nothing.

 Actions taken:  92
Reward:  (0, False)
Time: 92  seconds elapsed
TAMA STATS:  {'health': 57.0, 'hygiene': 8.0, 'happiness': 8.0, 'food': 8.0, 'energy': 32.0}
Tama is asleep.
Number of poo: 0
Money: 0
You're not in the store, so you can play, do nothing, or travel to the store.


Num sims run: 5817   Time elapsed: 0:00:03.000415
store: 0.00% (0 / 0)
play: 0.00% (0 / 0)
nothing: 0.00% (0 / 0)
Maximum depth search: 0

Taking action store.

 Actions taken:  93
Reward:  (0, False)
Time: 93  seconds elapsed
TAMA STATS:  {'health': 56.0, 'hygiene': 7.0, 

In [35]:
print(action_seq)

['store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing', 'store', 'nothing']

In [53]:
legal = ['coffee','snack','clean','nothing']
moves = [(state, a) for a in legal]
state.print_tama()
moves


Time: 1894  seconds elapsed
Tama is dead!
TAMA STATS:  {'food': 1.5277777777796062, 'energy': 55.972222222222236, 'happiness': 85.97222222221967, 'hygiene': 0, 'health': 0}
Tama is asleep.
Number of poo: 0
Money: 9
You're not in the store, so you can play, do nothing, or travel to the store.




[(<__main__.Tamagotchi at 0x10f771278>, 'coffee'),
 (<__main__.Tamagotchi at 0x10f771278>, 'snack'),
 (<__main__.Tamagotchi at 0x10f771278>, 'clean'),
 (<__main__.Tamagotchi at 0x10f771278>, 'nothing')]